In [1]:
# !/bin/python3
# https://adventofcode.com/2022/day/21

%load_ext lab_black

In [2]:
import json

from copy import deepcopy
from utils import read_input

In [3]:
def parse(input):
    data = {}
    for line in input:
        line = line.split(": ")
        name = line[0]
        if line[1].isnumeric():
            data[name] = int(line[1])
        else:
            data[name] = line[1].split(" ")
    return data

In [4]:
def add(data, a, b):
    a = get_value(data, a)
    b = get_value(data, b)
    return a + b


def subtract(data, a, b):
    a = get_value(data, a)
    b = get_value(data, b)
    return a - b


def multiply(data, a, b):
    a = get_value(data, a)
    b = get_value(data, b)
    return a * b


def divide(data, a, b):
    a = get_value(data, a)
    b = get_value(data, b)
    return a // b


operations = {"+": add, "-": subtract, "*": multiply, "/": divide}


def get_value(data, name):
    if type(name) == int:
        return name

    if type(data[name]) != list:
        return data[name]

    values = data[name]
    operation = operations[values[1]]
    a = values[0]
    b = values[2]

    result = operation(data, get_value(data, a), get_value(data, b))
    return result

In [5]:
def part_a(data):
    return get_value(data, "root")

In [6]:
def track(data, name, queue=[], has_humn=False):
    if type(name) == int or name == "humn":
        queue.append(name)
        return queue, has_humn

    if type(data[name]) == int:
        queue.append(data[name])
        return queue, has_humn

    values = data[name]
    if "humn" in values:
        has_humn = True

    operation = values[1]
    a = values[0]
    b = values[2]

    curr = [operation]
    curr, has_humn = track(data, a, queue=curr, has_humn=has_humn)
    curr, has_humn = track(data, b, queue=curr, has_humn=has_humn)

    queue.append(curr)
    return queue, has_humn


def get_opposite_operation(data, operation, parent, value, value_is_first=True):
    if operation == "*":
        return operations["/"](data, parent, value)

    if operation == "/":
        if value_is_first:
            return operations["/"](data, value, parent)
        return operations["*"](data, parent, value)

    if operation == "+":
        return operations["-"](data, parent, value)

    if operation == "-":
        if value_is_first:
            return operations["-"](data, value, parent)
        return operations["+"](data, parent, value)


def calculate(data, queue, parent=None):
    op = queue[0]
    a = queue[1]
    b = queue[2]

    if a == "humn":
        if type(b) == list:
            b = calculate(data, b)
        return get_opposite_operation(data, op, parent, b, value_is_first=False)

    if b == "humn":
        if type(a) == list:
            a = calculate(data, a)
        return get_opposite_operation(data, op, parent, a)

    if type(a) == list:
        if "humn" in json.dumps(a):
            if type(b) == list:
                b = calculate(data, b)
            if parent:
                b = get_opposite_operation(data, op, parent, b, value_is_first=False)
            return calculate(data, a, parent=b)
        a = calculate(data, a)

    if type(b) == list:
        if "humn" in json.dumps(b):
            if type(a) == list:
                a = calculate(data, a)
            if parent:
                a = get_opposite_operation(data, op, parent, a)
            return calculate(data, b, parent=a)
        b = calculate(data, b)

    return operations[op](data, a, b)

In [7]:
def part_b(data):
    del data["humn"]

    a, is_a_humn = track(data, data["root"][0], queue=[])
    b, is_b_humn = track(data, data["root"][2], queue=[])

    humn = a if is_a_humn else b
    humn = b if is_b_humn else a
    non_humn = a if not is_a_humn else b
    non_humn = b if not is_b_humn else a

    non_humn = calculate(data, non_humn[0])
    return calculate(data, humn[0], parent=non_humn)

In [8]:
data = parse(read_input(parent=__vsc_ipynb_file__, sample="a"))

print("part a:", part_a(deepcopy(data)))
print("part b:", part_b(data))

part a: 152
part b: 301


In [9]:
data = parse(read_input(parent=__vsc_ipynb_file__))

print("part a:", part_a(deepcopy(data)))
print("part b:", part_b(data))

part a: 157714751182692
part b: 3373767893067
